In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [2]:
# 총 2040개의 데이터
# 0:1992 -> train
# 1992:2016 -> valid
# 2016:2040 -> test

CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'super_basic',
    
    'test_lenght': 24,
    'valid_start_index': 1992,
    'test_start_index': 2016,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train.csv', encoding='cp949')

In [4]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, ['target']]

print(f'data.shape: {data.shape}')

data.shape: (2040, 1)


In [5]:
input_cols = [f't-{i}' for i in range(CONFIGS['window_size'], 0, -1)]
target_cols = [f't+{i}' for i in range(CONFIGS['target_length'])]

CONFIGS['input_cols'] = input_cols
CONFIGS['target_cols'] = target_cols

In [6]:
def mk_time_series(data):
    
    new_data_length = data.shape[0]-CONFIGS['window_size']-CONFIGS['target_length']+1
    new_data_shape = (new_data_length, CONFIGS['window_size']+CONFIGS['target_length'])
    new_data = np.zeros(new_data_shape)

    for i in range(new_data_length):
        new_data[i, :CONFIGS['window_size']] = data['target'][i:i+CONFIGS['window_size']]
        new_data[i, CONFIGS['window_size']:] = \
            data['target'][i+CONFIGS['window_size']:i+CONFIGS['window_size']+CONFIGS['target_length']]

    new_data = pd.DataFrame(new_data)
    new_data.columns = input_cols + target_cols
    
    return new_data

In [7]:
def split_data(data, CONFIGS):
    
    train = data[:CONFIGS['valid_start_index']]
    valid = data[
        CONFIGS['valid_start_index']-CONFIGS['window_size']:\
        CONFIGS['valid_start_index']+CONFIGS['test_lenght']
    ]
    test = data[
        CONFIGS['test_start_index']-CONFIGS['window_size']:\
        CONFIGS['test_start_index']+CONFIGS['test_lenght']
    ]
    
    train, valid, test = \
        mk_time_series(train), mk_time_series(valid), mk_time_series(test)
    
    return train, valid, test

In [8]:
train, valid, test = split_data(data, CONFIGS)

In [9]:
mse = MeanSquaredError()

In [10]:
def set_model(CONFIGS, model_name = None, print_summary=False):
    inputs = Input(batch_shape=(None, CONFIGS['window_size']), name='inputs')
    dense_0 = Dense(64, activation='relu', name='dense_0')(inputs)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dense_0)
    outputs = Dense(CONFIGS['target_length'], name='outputs')(dense_1)
    
    if not model_name:
        model_name = CONFIGS['model_name']
    
    model = Model(
        inputs, outputs,
        name = model_name
    )
    
    optimizer = Adam(learning_rate=CONFIGS['learning_rate'])
    model.compile(
        loss = mse,
        optimizer = optimizer,
    )
    
    if print_summary:
        model.summary()
    
    return model

In [11]:
model = set_model(CONFIGS, print_summary=True)

Model: "super_basic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 168)]             0         
_________________________________________________________________
dense_0 (Dense)              (None, 64)                10816     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
outputs (Dense)              (None, 3)                 99        
Total params: 12,995
Trainable params: 12,995
Non-trainable params: 0
_________________________________________________________________


In [12]:
def train_model(model, train, valid, CONFIGS):
    
    X_train, y_train = train[CONFIGS['input_cols']], train[CONFIGS['target_cols']]
    X_valid, y_valid = valid[CONFIGS['input_cols']], valid[CONFIGS['target_cols']]
    
    early_stop = EarlyStopping(
        patience=CONFIGS['es_patience']
    )
    save_best_only = ModelCheckpoint(
        filepath = f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5',
        monitor = 'val_loss',
        save_best_only = True,
        save_weights_only = True
    )
    
    history = model.fit(
        X_train, y_train,
        batch_size = CONFIGS['batch_size'],
        epochs = CONFIGS['epochs'],
        validation_data = (X_valid, y_valid),
        callbacks = [
            early_stop,
            save_best_only,
        ]
    )
    
    return history

In [13]:
history = train_model(model, train, valid, CONFIGS)

Epoch 1/100
29/29 [==============================] - 1s 5ms/step - loss: 73326104.0000 - val_loss: 29656152.0000
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 14798467.0000 - val_loss: 4228784.0000
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 1994319.7500 - val_loss: 488892.9062
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 196021.2500 - val_loss: 29830.4297
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 17347.6445 - val_loss: 10966.3320
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 8371.7090 - val_loss: 15605.4854
Epoch 7/100
29/29 [==============================] - 0s 2ms/step - loss: 7508.0249 - val_loss: 16568.9609
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 7443.3921 - val_loss: 16937.6895
Epoch 9/100
29/29 [==============================] - 0s 2ms/step - loss: 7440.6514 - val_loss: 16730.9688
Epoch 10/100
29/29 [======

In [14]:
best_model = set_model(CONFIGS, model_name='best_'+CONFIGS['model_name'])
best_model.load_weights(f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5')

In [15]:
X_train, y_train = train[CONFIGS['input_cols']], train[CONFIGS['target_cols']]
X_valid, y_valid = valid[CONFIGS['input_cols']], valid[CONFIGS['target_cols']]
X_test, y_test = test[CONFIGS['input_cols']], test[CONFIGS['target_cols']]

y_train_pred = best_model.predict(X_train)
y_valid_pred = best_model.predict(X_valid)
y_test_pred = best_model.predict(X_test)

In [16]:
train_loss = mse(y_train, y_train_pred).numpy()
valid_loss = mse(y_valid, y_valid_pred).numpy()
test_loss = mse(y_test, y_test_pred).numpy()

print(f'train_loss: {train_loss}')
print(f'valid_loss: {valid_loss}')
print(f'test_loss: {test_loss}')

train_loss: 9796.9130859375
valid_loss: 10966.33203125
test_loss: 14396.3349609375
